In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain import SQLDatabase
from langchain.chains import create_sql_query_chain
import os 
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from langchain.llms import AzureOpenAI

In [2]:
from langchain.chat_models import AzureChatOpenAI

In [3]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://ai-stage2.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] = '2023-08-01-preview'
os.environ["OPENAI_API_KEY"] = "cdf63ece486a426da741e31ef181950c"

In [4]:
url = URL(
    user='itsmejamshid',
    password='Mucklab17',
    account='ln12766.uae-north.azure',
    warehouse='COMPUTE_WH',
    database='TESTFORADF',
    schema='PUBLIC',
    role = 'ACCOUNTADMIN'
)
engine = create_engine(url)

In [5]:
connection = engine.connect()

In [6]:
db = SQLDatabase(engine, include_tables=["balance", "cash", "investment", "turnover"])


In [7]:
db.get_usable_table_names()

['balance', 'cash', 'investment', 'turnover']

In [8]:
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


In [9]:
# SQLDatabaseToolkit.get_tools('')

In [10]:
toolkit = SQLDatabaseToolkit(db=db, llm=AzureChatOpenAI(temperature=0))

In [11]:
PROMPT_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Since you are working with snowflake, you have to get table names without the quotes.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
If user asks you to append multiple tables or to produce output table from multiple tables, you have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
Since all of the columns in the tables are distinct and do not share common name, you have to use the UNION function in sql query to merge tables into single output table by appending them.
If there are multiple tables, since the columns in the tables are different, we will need to fill in the missing columns with NULL values to make the number of columns equal.
Before producing any sql query to create output table, you should ask what would be the name for the output table name from user.
When there are common column names among the tables, you need make them distinctive by merging table's name and column name to create new distinctive name.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer."""

In [24]:
toolkit.get_tools()[0]("Select * from 'balance'")

QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain.utilities.sql_database.SQLDatabase object at 0x160335d80>)

In [12]:
agent_executor = create_sql_agent(
    llm=AzureChatOpenAI(deployment_name="gpt-4", temperature=0),
    toolkit=toolkit,
    prefix=PROMPT_PREFIX,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [13]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct snowflake query to run, then look at the results of the query and return the answer.
Since you are working with snowflake, you have to get table names without the quotes.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10\ results.
You can order the results by a relevant column to return the most interesting examples in the database.
If user asks you to append multiple tables or to produce output table from multiple tables, you have to append multiple tables so that output must contain all of the columns in the tables user mentioned. 
Since all of the columns in the tables are distinct and do not share common name, you have to use the UNION function in sql query to merge tables into single output table by appending them.
If there are multiple tables, since the columns in the tables are different, we will need 

In [17]:
agent_executor.run("describe balance table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ''
Observation: balance, cash, investment, turnover
Thought:The balance table is in the database. I should now query the schema of the balance table.
Action: sql_db_schema
Action Input: 'balance'
Observation: Error: table_names {"'balance'"} not found in database
Thought:It seems like there was a mistake with the table name. The table names should not be enclosed in quotes.
Action: sql_db_schema
Action Input: balance
Observation: 
CREATE TABLE balance (
	"Fund Name" VARCHAR(200), 
	"Receivables" FLOAT, 
	"Liabilities" FLOAT, 
	"Total Net Assets" FLOAT, 
	"Date" VARCHAR(200)
)

/*
3 rows from balance table:
Fund Name	Receivables	Liabilities	Total Net Assets	Date
1736917 - LCIV GB ALPHA GROWTH FND	18883208.64	-14815065.4	2730435768.31	30 Sep 21
1736938 - LCIV DIVERSIFD GROWTH FND	0.0	-791115.69	694714006.32	30 Sep 21
1740040 - LCIV GB TOTAL RETURN FND	0.0	-287602.49	244329466.82	30 Sep 21
*/
Thought:I now kn

'The balance table has the following columns: "Fund Name", "Receivables", "Liabilities", "Total Net Assets", and "Date". Here are some sample rows from the balance table:\n\n1. Fund Name: 1736917 - LCIV GB ALPHA GROWTH FND, Receivables: 18883208.64, Liabilities: -14815065.4, Total Net Assets: 2730435768.31, Date: 30 Sep 21\n2. Fund Name: 1736938 - LCIV DIVERSIFD GROWTH FND, Receivables: 0.0, Liabilities: -791115.69, Total Net Assets: 694714006.32, Date: 30 Sep 21\n3. Fund Name: 1740040 - LCIV GB TOTAL RETURN FND, Receivables: 0.0, Liabilities: -287602.49, Total Net Assets: 244329466.82, Date: 30 Sep 21'

In [97]:
agent_executor.run("describe all of the tables")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ''
Observation: balance, cash, investment, turnover
Thought:Now that I have the list of tables, I should get the schema for each of these tables.
Action: sql_db_schema
Action Input: 'balance, cash, investment, turnover'
Observation: Error: table_names {"'balance", "turnover'"} not found in database
Thought:It seems like there was an error with the table names. I should try again with the correct table names.
Action: sql_db_schema
Action Input: 'balance, cash, investment, turnover'
Observation: Error: table_names {"'balance", "turnover'"} not found in database
Thought:It seems like there was an error with the table names. I should try again with the correct table names.
Action: sql_db_schema
Action Input: 'balance, cash, investment, turnover'
Observation: Error: table_names {"'balance", "turnover'"} not found in database
Thought:It seems like there was an error with the table names. I should try again with 

KeyboardInterrupt: 